# `mesh_mask202108.nc` Metadata

Add metadata to the NEMO-generated mesh mask file for the 202108 bathymetry
so that a well-defined ERDDAP dataset can be produced from it.

In [1]:
from datetime import datetime

import xarray as xr


In [2]:
repo_file = "grid/mesh_mask202108.nc"
mm = xr.open_dataset(f"../../{repo_file}")


In [3]:
# Copy/pasted from Google Drive MeshMask Variables spreadsheet

# NEMO Variable Name	Description -> long_name Attribute	Units
nemo_vars = '''
e1t	grid spacing on T-grid in u direction	m
e2t	grid spacing on T-grid in v direction	m
e1u	grid spacing on U-grid in u direction	m
e2u 	grid spacing on U-grid in v direction	m
e1v	grid spacing on V-grid in u direction	m
e2v	grid spacing on V-grid in v direction	m
e1f	grid spacing on vorticity-grid in u direction	m
e2f	grid spacing on vorticity-grid in v direction	m
glamt	longitude of T-grid points	degrees east
gphit	latitude of T-grid points	degrees north
glamu	longitude of U-grid points	degrees east
gphiu	latitude of U-grid points	degrees north
glamv	longitude of V-grid points	degrees east
gphiv	latitude of V-grid points	degrees north
glamf	longitude of vorticity-grid points	degrees east
gphif	latitude of vorticity-grid points	degrees north
tmaskutil	dry land mask for T-grid and W-grid	boolean
umaskutil	dry land mask for U-grid	boolean
vmaskutil	dry land mask for V-grid	boolean
fmaskutil	dry land mask for vorticity-grid	boolean
ff	Coriolis parameter on vorticity-grid	s-1
mbathy	fortran index of deepest water cell, T-grid	index
e3t_0	grid spacing on T-grid in w direction	m
e3u_0	grid spacing on U-grid in w direction	m
e3v_0	grid spacing on V-grid in w direction	m
e3w_0	grid spacing on W-grid in w direction	m
e3t_1d	nominal vertical grid spacing on T-grid	m
e3w_1d	nominal vertical grid spacing on W-grid	m
gdept_0	depth of T-grid points	m
gdepu	depth of U-grid points	m
gdepv	depth of V-grid points	m
gdepw_0	depth of W-grid points	m
gdept_1d	nominal depth of T-grid points	m
gdepw_1d	nominal depth of W-grid points	m
tmask	mask for T-grid and W-grid	boolean
umask	mask for U-grid	boolean
vmask	mask for V-grid	boolean
fmask	mask for vorticity-grid	boolean
misf	fortran index of shallowest water, T-grid	index
isfdraft	thickness of land-fast ice below sea level	m
nav_lat	latitude of T-grid points	degrees north
nav_lon	longitude of T-grid points	degrees east
time_counter	time	seconds since 1900-01-01 00:00:00
'''


In [4]:
def interesting_lines(nemo_vars):
    for line in nemo_vars.splitlines():
        if line:
            yield line
            
units_subs = {
    'index': 'count',
    'degrees east': 'degrees_east',
    'degrees north': 'degrees_north',
}

for line in interesting_lines(nemo_vars):
    var_name, long_name, units = map(str.strip, line.split('\t'))
    mm[var_name].attrs['standard_name'] = var_name
    mm[var_name].attrs['long_name'] = long_name
    if units == 'boolean':
        mm[var_name].attrs['flag_values'] = '0, 1'
        mm[var_name].attrs['flag_meanings'] = 'land, water'
    else:
        try:
            mm[var_name].attrs['units'] = units_subs[units]
        except KeyError:
            mm[var_name].attrs['units'] = units
    if 'depth' in long_name:
        mm[var_name].attrs['positive'] = 'down'
    if 'time' in long_name:
        mm[var_name].attrs['calendar'] = 'gregorian'
        mm[var_name].attrs['time_origin'] = '2007-01-01 00:30:00'
        
del mm['nav_lev']


Spot check results:

In [5]:
mm


<xarray.Dataset>
Dimensions:       (y: 898, x: 398, t: 1, z: 40)
Dimensions without coordinates: y, x, t, z
Data variables: (12/43)
    nav_lon       (y, x) float32 ...
    nav_lat       (y, x) float32 ...
    time_counter  (t) float64 ...
    tmask         (t, z, y, x) int8 ...
    umask         (t, z, y, x) int8 ...
    vmask         (t, z, y, x) int8 ...
    ...            ...
    gdepv         (t, z, y, x) float32 ...
    gdepw_0       (t, z, y, x) float32 ...
    gdept_1d      (t, z) float64 ...
    gdepw_1d      (t, z) float64 ...
    e3t_1d        (t, z) float64 ...
    e3w_1d        (t, z) float64 ...
Attributes:
    file_name:  mesh_mask.nc
    TimeStamp:  07/08/2021 17:25:49 -0700

In [6]:
mm.time_counter


<xarray.DataArray 'time_counter' (t: 1)>
[1 values with dtype=float64]
Dimensions without coordinates: t
Attributes:
    standard_name:  time_counter
    long_name:      time
    units:          seconds since 1900-01-01 00:00:00
    calendar:       gregorian
    time_origin:    2007-01-01 00:30:00

In [7]:
mm.e1t


<xarray.DataArray 'e1t' (t: 1, y: 898, x: 398)>
[357404 values with dtype=float64]
Dimensions without coordinates: t, y, x
Attributes:
    standard_name:  e1t
    long_name:      grid spacing on T-grid in u direction
    units:          m

In [8]:
mm.glamt


<xarray.DataArray 'glamt' (t: 1, y: 898, x: 398)>
[357404 values with dtype=float32]
Dimensions without coordinates: t, y, x
Attributes:
    standard_name:  glamt
    long_name:      longitude of T-grid points
    units:          degrees_east

In [9]:
mm.gphit


<xarray.DataArray 'gphit' (t: 1, y: 898, x: 398)>
[357404 values with dtype=float32]
Dimensions without coordinates: t, y, x
Attributes:
    standard_name:  gphit
    long_name:      latitude of T-grid points
    units:          degrees_north

In [10]:
mm.gdept_0


<xarray.DataArray 'gdept_0' (t: 1, z: 40, y: 898, x: 398)>
[14296160 values with dtype=float32]
Dimensions without coordinates: t, z, y, x
Attributes:
    standard_name:  gdept_0
    long_name:      depth of T-grid points
    units:          m
    positive:       down

In [11]:
mm.tmask


<xarray.DataArray 'tmask' (t: 1, z: 40, y: 898, x: 398)>
[14296160 values with dtype=int8]
Dimensions without coordinates: t, z, y, x
Attributes:
    standard_name:  tmask
    long_name:      mask for T-grid and W-grid
    flag_values:    0, 1
    flag_meanings:  land, water

In [12]:
mm.ff


<xarray.DataArray 'ff' (t: 1, y: 898, x: 398)>
[357404 values with dtype=float64]
Dimensions without coordinates: t, y, x
Attributes:
    standard_name:  ff
    long_name:      Coriolis parameter on vorticity-grid
    units:          s-1

In [13]:
mm.mbathy


<xarray.DataArray 'mbathy' (t: 1, y: 898, x: 398)>
[357404 values with dtype=int16]
Dimensions without coordinates: t, y, x
Attributes:
    standard_name:  mbathy
    long_name:      fortran index of deepest water cell, T-grid
    units:          count

Update dataset attributes:

In [14]:
mm.attrs['file_name'] = repo_file
mm_timestamp = datetime.strptime(mm.attrs['TimeStamp'], '%d/%m/%Y %H:%M:%S %z')
mm.attrs['Conventions'] = 'CF-1.6'
mm.attrs['title'] = 'SalishSeaCast NEMO bathymetry_202108 Bathymetry Mesh Mask'
mm.attrs['institution'] = 'Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia'
mm.attrs['source'] = 'NEMO-3.6 SalishSeaCast configuration'
mm.attrs['references'] = "https://github.com/SalishSeaCast/tools/blob/main/bathymetry/Process202108Bathymetry.ipynb"
mm.attrs['history'] = ('''[{0:%Y-%m-%d %H:%M}] ncks -4 -L4 -O mesh_mask.nc mesh_mask202108.nc
[{1:%Y-%m-%d %H:%M}] Added metadata to variables in preparation for creation of ERDDAP datasets.'''
.format(mm_timestamp, datetime.now()))


In [15]:
mm.attrs


{'file_name': 'grid/mesh_mask202108.nc',
 'TimeStamp': '07/08/2021 17:25:49 -0700',
 'Conventions': 'CF-1.6',
 'title': 'SalishSeaCast NEMO bathymetry_202108 Bathymetry Mesh Mask',
 'institution': 'Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia',
 'source': 'NEMO-3.6 SalishSeaCast configuration',
 'references': 'https://github.com/SalishSeaCast/tools/blob/main/bathymetry/Process202108Bathymetry.ipynb',
 'history': '[2021-08-07 17:25] ncks -4 -L4 -O mesh_mask.nc mesh_mask202108.nc\n[2023-12-19 13:36] Added metadata to variables in preparation for creation of ERDDAP datasets.'}

In [16]:
encoding = {var: {'zlib': True} for var in mm.data_vars}
mm.to_netcdf('foo.nc', engine='netcdf4', format='netcdf4', encoding=encoding)
